# ETAPA 1 - Preparação dos Dados

### #=====Vizualização dos Dados======#

> **Nota:** Toda a preparação dos dados também está disponível no script `src/data_preprocessing.py`, com a mesma lógica descrita aqui.  
> No entanto, optamos por documentar e rodar tudo diretamente neste notebook (`olist_analysis.ipynb`) para fins de explicação detalhada e rastreabilidade.

Nesta etapa, foi realizada a leitura de todos os arquivos CSV da base de dados do Olist, localizados na pasta data/raw/.
Utilizei um dicionário (file_map) para mapear nomes intuitivos aos nomes dos arquivos. Em seguida, carreguei os dados com pandas.read_csv() e armazenamos os DataFrames em um dicionário (dfs) para facilitar o acesso e manipulação. Por fim, foi exibida as primeiras linhas de cada tabela para verificar se os dados foram carregados corretamente.

In [ ]:
import pandas as pd
import os

#===Definindo o caminho dos arquivos brutos===#
raw_path = '../data/raw/'

#===Mapeando os arquivos em um dicionário para facilitar o carregamento===#
file_map = {
    "orders": "olist_orders_dataset.csv",
    "customers": "olist_customers_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "order_reviews": "olist_order_reviews_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "product_translation": "product_category_name_translation.csv"
}

#===Carregando todos os arquivos em dataframes===#
dfs = {name: pd.read_csv(os.path.join(raw_path, filename)) for name, filename in file_map.items()}

#===Exibindo os primeiros registros de cada dataset===#
for name, df in dfs.items():
    print(f"\n{name.upper()}:\n")
    display(df.head())

### #======Limpeza e Normalização dos Dados======#

Nesta etapa, apliquei duas funções auxiliares para padronizar e limpar todos os DataFrames carregados:

- normalize_columns(): transforma os nomes das colunas para minúsculo, substitui espaços e hífens por underscores (_), deixando o padrão mais limpo e consistente.

- clean_df(): remove linhas duplicadas, linhas e colunas totalmente vazias.

Após aplicar essas funções a todos os DataFrames, salvei os arquivos limpos na pasta data/processed/, organizando o fluxo de dados para etapas futuras da análise.

In [ ]:
#===Função que normaliza nomes de colunas===#
def normalize_columns(df):
    return df.rename(columns=lambda x: x.strip().lower().replace(" ", "_").replace("-", "_"))

#===Função de limpeza básica===#
def clean_df(df):
    df = df.drop_duplicates()           #->Remove duplicatas
    df = df.dropna(how='all')           #->Remove linhas totalmente vazias
    df = df.dropna(axis=1, how='all')   #->Remove colunas totalmente vazias
    return df

#===Aplicando limpeza e normalização em todos os dataframes===#
for name in dfs:
    dfs[name] = normalize_columns(dfs[name])
    dfs[name] = clean_df(dfs[name])

#===Correções específicas para colunas com erros de digitação===#
dfs["products"].rename(columns={
    "product_name_lenght": "product_name_length",
    "product_description_lenght": "product_description_length"
}, inplace=True)

#===Salvando os dados limpos em CSVs===#
processed_path = "../data/processed/"
if os.path.exists(processed_path) and not os.path.isdir(processed_path):
    os.remove(processed_path)
os.makedirs(processed_path, exist_ok=True)

for name, df in dfs.items():
    df.to_csv(os.path.join(processed_path, f"{name}.csv"), index=False)

### #======Criação do banco de dados relacional======#

- Defini as tabelas com chaves primárias e estrangeiras, respeitando as relações entre os dados (ex: order_items referenciando orders, products e sellers).

- Inseri todos os dados do dicionário dfs diretamente nas tabelas do banco, utilizando pandas.to_sql() com if_exists="append".

In [ ]:
import sqlite3
import os

#===Caminho para salvar o banco SQLite===#
processed_path = "../data/processed"
db_path = os.path.join(processed_path, "olist_ecommerce.db")

#===Conexão com o banco===#
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

#===Criação das tabelas com chaves primárias e estrangeiras===#
cursor.executescript("""
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS sellers;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS order_items;
DROP TABLE IF EXISTS order_reviews;
DROP TABLE IF EXISTS order_payments;
DROP TABLE IF EXISTS geolocation;
DROP TABLE IF EXISTS product_translation;

CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix INTEGER,
    customer_city TEXT,
    customer_state TEXT
);

CREATE TABLE sellers (
    seller_id TEXT PRIMARY KEY,
    seller_zip_code_prefix INTEGER,
    seller_city TEXT,
    seller_state TEXT
);

CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_name_length REAL,
    product_description_length REAL,
    product_photos_qty REAL,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL
);

CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
);

CREATE TABLE order_reviews (
    review_id TEXT PRIMARY KEY,
    order_id TEXT,
    review_score INTEGER,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date TEXT,
    review_answer_timestamp TEXT,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE order_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE geolocation (
    geolocation_zip_code_prefix INTEGER,
    geolocation_lat REAL,
    geolocation_lng REAL,
    geolocation_city TEXT,
    geolocation_state TEXT
);

CREATE TABLE product_translation (
    product_category_name TEXT PRIMARY KEY,
    product_category_name_english TEXT
);
""")

#===Removendo duplicatas com base nas chaves primárias antes da inserção===#
dfs["customers"].drop_duplicates(subset=["customer_id"], inplace=True)
dfs["sellers"].drop_duplicates(subset=["seller_id"], inplace=True)
dfs["products"].drop_duplicates(subset=["product_id"], inplace=True)
dfs["orders"].drop_duplicates(subset=["order_id"], inplace=True)
dfs["order_reviews"].drop_duplicates(subset=["review_id"], inplace=True)
dfs["order_items"].drop_duplicates(subset=["order_id", "order_item_id"], inplace=True)
dfs["product_translation"].drop_duplicates(subset=["product_category_name"], inplace=True)

# Inserção dos dados em cada tabela
for name, df in dfs.items():
    df.to_sql(name, conn, if_exists="append", index=False)

conn.commit()
conn.close()

### #======Verificação do Banco de Dados======#

Listei todas as tabelas criadas no SQLite para garantir que foram geradas corretamente.

Também executei um JOIN entre orders, customers e order_payments para validar os relacionamentos e conferir se os dados estão integrados como esperado.

In [ ]:
#===Conectando ao banco SQLite===#
import sqlite3

conn = sqlite3.connect("../data/processed/olist_ecommerce.db")

#===Conferindo se as tabelas foram criadas no banco===#
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

#===Validando relacionamento com SQL JOIN===#
query = """
SELECT 
    o.order_id,
    o.order_purchase_timestamp,
    c.customer_state,
    p.payment_value
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_payments p ON o.order_id = p.order_id
LIMIT 5;
"""

pd.read_sql(query, conn)

# ETAPA 2 - Análise Exploratória de Dados

### (a) Qual o volume de pedidos por mês? Existe sazonalidade nas vendas?

- Extrair o mês (%Y-%m) da data de compra (order_purchase_timestamp)
- Contar o número de pedidos agrupados por mês
- Plotar a série temporal para visualizar padrões de crescimento e sazonalidad

In [ ]:
#===Conectando ao banco SQLite onde estão os dados organizados===#
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/processed/olist_ecommerce.db")  #->Abre conexão com o banco de dados

#===Consulta SQL para contar quantos pedidos foram feitos por mês===#
query = """
SELECT
    strftime('%Y-%m', order_purchase_timestamp) AS order_month, 
    COUNT(order_id) AS total_orders                             
FROM orders
GROUP BY order_month
ORDER BY order_month;
"""

#===Lê o resultado da query e transforma em DataFrame===#
df_orders_by_month = pd.read_sql(query, conn)

#===Visualizando os primeiros resultados===#
df_orders_by_month.head()

Com a query acima, os pedidos serão agrupaddos por mês utilizando a função `strftime('%Y-%m', ...)`.  
Esse agrupamento permitirá identificar flutuações mensais e possíveis padrões sazonais nas vendas.  
Abaixo, segue um  gráfico de linha básico com seaborn para identificar essas flutuações e possíveis padrões nas vendas.

In [ ]:
#===Importando bibliotecas para visualização===#
import matplotlib.pyplot as plt
import seaborn as sns

#===Criando gráfico de linha para visualizar a evolução dos pedidos mês a mês===#
plt.figure(figsize=(14,6))
sns.lineplot(data=df_orders_by_month, x="order_month", y="total_orders", marker="o")  #->Gráfico de linha com marcação nos pontos
plt.xticks(rotation=45)                                                                #->Rotaciona os rótulos do eixo x
plt.title("Volume de Pedidos por Mês")                                                 #->Título do gráfico
plt.xlabel("Mês")
plt.ylabel("Número de Pedidos")
plt.tight_layout()
plt.show()

### (b) Qual a distribuição do tempo de entrega dos pedidos?

- Calcular a diferença entre a data de entrega (order_delivered_customer_date) e a data da compra (order_purchase_timestamp)
- Converter para número de dias
- Visualizar com histograma para entender os tempos típicos e outliers

In [ ]:
#===Consulta SQL para pegar a data da compra e da entrega do pedido===#
query = """
SELECT 
    order_id,
    order_purchase_timestamp,
    order_delivered_customer_date
FROM orders
WHERE order_delivered_customer_date IS NOT NULL
AND order_purchase_timestamp IS NOT NULL;
"""

#===Executando a query e armazenando o resultado em um DataFrame===#
df_delivery = pd.read_sql(query, conn)

#===Convertendo as colunas de data para o formato datetime do pandas===#
df_delivery['order_purchase_timestamp'] = pd.to_datetime(df_delivery['order_purchase_timestamp'])
df_delivery['order_delivered_customer_date'] = pd.to_datetime(df_delivery['order_delivered_customer_date'])

#===Calculando o tempo de entrega em dias===#
df_delivery['delivery_time_days'] = (df_delivery['order_delivered_customer_date'] - df_delivery['order_purchase_timestamp']).dt.days

#===Visualizando os primeiros valores da nova coluna criada===#
df_delivery[['order_id', 'delivery_time_days']].head()

A distribuição mostra o número de dias entre a compra e a entrega dos pedidos.  
Esse dado será útil para a análise de predição de atrasos (Etapa 3) e também para investigar a satisfação dos clientes.  
No gráfico abaixo, está um histograma com curva de densidade que ilustra claramente essa distribuição de tempos de entrega.

In [ ]:
#===Visualizando a distribuição dos tempos de entrega com histograma===#
plt.figure(figsize=(10,6))
sns.histplot(df_delivery['delivery_time_days'], bins=30, kde=True)  #->Histograma com 30 faixas e curva de densidade
plt.title("Distribuição do Tempo de Entrega dos Pedidos (em dias)")
plt.xlabel("Dias")
plt.ylabel("Frequência")
plt.tight_layout()

### (c) Qual a relação entre o valor do frete e a distância de entrega? 
- Pegar as médias de lat/lng por zip_code_prefix
- Juntar customers + sellers + order_items
- Calcular a distância com a fórmula de Haversine
- Ver a relação entre freight_value e distância

**Desafios de performance:**
- A tabela geolocation tem dados duplicados por CEP, então precisamos agrupar
- A fórmula de Haversine é pesada, então vamos aplicar só depois do SELECT
- Para acelerar tudo, vamos criar uma VIEW e índices no banco SQLite

#### Criando uma VIEW no banco para calcular a média de latitude/longitude por CEP

Em vez de calcular a média de coordenadas geográficas toda vez que rodar a query, criei uma VIEW chamada geo_avg.  
Ela já deixa os dados prontos e agrupados por `zip_code_prefix`, facilitando os JOINs e melhorando o desempenho.


In [ ]:
#===Criação de VIEW: média da latitude e longitude por CEP para evitar recalcular isso várias vezes===#
cursor = conn.cursor()

cursor.executescript("""
DROP VIEW IF EXISTS geo_avg;
CREATE VIEW geo_avg AS
SELECT 
    geolocation_zip_code_prefix,
    AVG(geolocation_lat) AS lat,
    AVG(geolocation_lng) AS lng
FROM geolocation
GROUP BY geolocation_zip_code_prefix;
""")
conn.commit()


#### Criando índices para acelerar os JOINs

Índices froma criados nas colunas que participam de JOINs nas tabelas principais.  
Isso melhora a velocidade de resposta, especialmente com conjuntos de dados maiores.


In [ ]:
#===Criando índices para acelerar os JOINs nas colunas mais usadas===#
cursor.executescript("""
CREATE INDEX IF NOT EXISTS idx_customer_zip ON customers(customer_zip_code_prefix);
CREATE INDEX IF NOT EXISTS idx_seller_zip ON sellers(seller_zip_code_prefix);
CREATE INDEX IF NOT EXISTS idx_geo_zip ON geolocation(geolocation_zip_code_prefix);
CREATE INDEX IF NOT EXISTS idx_order_customer ON orders(customer_id);
CREATE INDEX IF NOT EXISTS idx_order_item_order ON order_items(order_id);
CREATE INDEX IF NOT EXISTS idx_order_item_seller ON order_items(seller_id);
""")
conn.commit()


#### Rodando a query para coletar frete e coordenadas de cliente e vendedor

Todos os JOINs necessários com a geo_avg para pegar a posição média do cliente e do vendedor.  
Aplicando tambeém um LIMIT de 5000 linhas e um filtro para o estado de SP para testar com mais rapidez.


In [ ]:
#===Query otimizada: JOIN com a VIEW geo_avg ao invés de recalcular os AVG de coordenadas===#
query = """
SELECT 
    o.order_id,
    o.seller_id,
    o.freight_value,
    gc.lat AS customer_lat,
    gc.lng AS customer_lng,
    gs.lat AS seller_lat,
    gs.lng AS seller_lng
FROM order_items o
JOIN orders ord ON o.order_id = ord.order_id
JOIN customers c ON ord.customer_id = c.customer_id
JOIN sellers s ON o.seller_id = s.seller_id
JOIN geo_avg gc ON c.customer_zip_code_prefix = gc.geolocation_zip_code_prefix
JOIN geo_avg gs ON s.seller_zip_code_prefix = gs.geolocation_zip_code_prefix
WHERE c.customer_state = 'SP'
LIMIT 5000;
"""

#===Executando a consulta e armazenando no DataFrame===#
df_dist = pd.read_sql(query, conn)


#### Calculando a distância entre cliente e vendedor

Calculei a distância entre cliente e vendedor utilizando a fórmula de Haversine que, baseado em trigonometria esfériaca, serve para estima a distância entre dois pontos na superfície de uma esfera, levando em consideração a curvatura da Terra.  
A aplicação foi feita linha a linha no DataFrame usando .apply() do pandas.

In [ ]:
#===Fórmula de Haversine: calcula a distância entre dois pontos do globo===#
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  #-> raio da Terra em km
    dlat = radians(lat2 - lat1) #-> Converte graus para radianos
    dlon = radians(lon2 - lon1)

    #===Fórmula de Haversine===#
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

#=== Criação da coluna com a distância estimada em km===#
df_dist['distance_km'] = df_dist.apply(
    lambda row: haversine(
        row["customer_lat"], row["customer_lng"], 
        row["seller_lat"], row["seller_lng"]),
    axis=1
)


A visualização sugere que há uma correlação entre a distância e o valor do frete.  
Para distâncias maiores, o frete tende a ser mais alto, o que é esperado.  
Contudo, há dispersão indicando que outros fatores também afetam o custo de entrega (peso, cidade, etc.).

In [ ]:
#===Criando gráfico para visualizar a relação entre frete e distância===#
plt.figure(figsize=(10,6))
sns.scatterplot(data=df_dist, x="distance_km", y="freight_value", alpha=0.5)
plt.title("Relação entre Distância e Valor do Frete")
plt.xlabel("Distância estimada (km)")
plt.ylabel("Frete (R$)")
plt.tight_layout()
plt.show()

###  (d) Categorias de produtos mais vendidas em faturamento
- Juntar order_items, products e product_category_name_translation
- Somar o valor total (price) vendido por categoria
- Ordenar do maior para o menor faturamento
- Visualizar as top 10 com gráfico de barras

In [ ]:
#===Consulta para somar o valor total vendido por categoria (já traduzida)===#
query = """
SELECT 
    pt.product_category_name_english AS category,
    SUM(oi.price) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN product_translation pt ON p.product_category_name = pt.product_category_name
GROUP BY category
ORDER BY total_revenue DESC
LIMIT 10;
"""

#===Lendo o resultado da query===#
df_revenue = pd.read_sql(query, conn)

In [ ]:
#===Criando gráfico de barras com as categorias que mais geram faturamento===#
plt.figure(figsize=(10,6))
sns.barplot(data=df_revenue, x="total_revenue", y="category")
plt.title("Top 10 Categorias por Faturamento")
plt.xlabel("Faturamento (R$)")
plt.ylabel("Categoria")
plt.tight_layout()
plt.show()

### (e) Estados brasileiros com maior valor médio de pedido
- Juntar orders, order_items e customers
- Agrupar por customer_state
- Calcular a média dos valores dos pedidos (price)
- Plotar o valor médio por estado com gráfico de barras

In [ ]:
#===Consulta SQL para calcular o valor médio dos pedidos por estado do cliente===#
query = """
SELECT 
    c.customer_state AS state,
    AVG(oi.price) AS avg_order_value
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY state
ORDER BY avg_order_value DESC;
"""

#===Executando e lendo os dados===#
df_avg_order = pd.read_sql(query, conn)

In [ ]:
#===Criando gráfico mostrando o valor médio por estado brasileiro===#
plt.figure(figsize=(10,6))
sns.barplot(data=df_avg_order, x="avg_order_value", y="state")
plt.title("Valor Médio de Pedido por Estado")
plt.xlabel("Valor Médio (R$)")
plt.ylabel("Estado")
plt.tight_layout()
plt.show()


# ETAPA 3 - Solução de Problemas de Negócio

### Subetapa 1: Análise de Retenção  
Nesta parte, foi calculado a taxa de clientes recorrentes. Considerando que um cliente recorrente é aquele que fez mais de um pdedio no período analisado  
- Agrupar os pedidos por customer_unique_id
- Contar quantos pedidos cada cliente fez
- Criar uma variável que marca se o cliente é recorrente (mais de 1 pedido)
- Calcular a taxa de recorrência (% de clientes que voltaram a comprar)

In [ ]:
#===Consulta SQL: agrupando os pedidos por cliente_unique_id===#
query = """
SELECT 
    c.customer_unique_id,
    COUNT(o.order_id) AS num_orders
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_unique_id;
"""

#===Executando a query e transformando em DataFrame===#
df_retention = pd.read_sql(query, conn)


In [ ]:
#===Criando coluna que indica se o cliente fez mais de um pedido===#
df_retention['is_recurring'] = df_retention['num_orders'] > 1  #->True se o cliente tiver feito mais de 1 pedido

#===Calculando a taxa de clientes recorrentes===#
recurring_rate = df_retention['is_recurring'].mean()  #->Média de valores True = % de recorrentes
total_clients = df_retention.shape[0]                  #->Total de clientes únicos
recurring_clients = df_retention['is_recurring'].sum() #->Quantidade de recorrentes (True)

#===Exibindo os resultados===#
print(f"Clientes totais: {total_clients}")
print(f"Clientes recorrentes: {recurring_clients}")
print(f"Taxa de recorrência: {recurring_rate:.2%}")


### Principais insights - Análise de Retenção
#### A maioria dos clientes compra apenas uma vez
- A taxa de recorrência é baixa, visto o resultado obtido após a consulta, o que mostra que quase todos os clientes fizeram apenas um pedido.
- Isso pode indicar que o cliente não tem motivos suficientes para voltar, seja por falta de incentivo, necessidade ou hábito.
#### O negócio pode estar dependendo de novos clientes
- Como poucos voltam a comprar, o crescimento pode estar baseado em atrair novos clientes o tempo todo, o que geralmente custa mais caro do que manter quem já comprou.
### Há espaço para melhorar a fidelização
- Se mais clientes fossem incentivados a comprar novamente, as vendas totais poderiam aumentar, mesmo com menos investimento em propaganda.
- Algumas ideias seriam: descontos para quem já comprou, envio de e-mails com ofertas personalizadas, ou programas simples de benefícios.




### Subetapa 2: Predição de Atraso

Aqui vamos criei um modelo simples para prever se um pedido será entregue com atraso.

- Considerar atrasado se a data real de entrega > data estimada
- Usar variáveis relacionadas ao valor do pedido, pagamento, frete e distância
- Treinar um modelo simples com Regressão Logística
- Corrigir o desbalanceamento das classes com `class_weight='balanced'`
- Avaliar os resultados com métricas de classificação


In [ ]:
#===Consulta SQL: dados do pedido, frete e pagamento===#
query = """
SELECT 
    o.order_id,
    o.order_purchase_timestamp,
    o.order_estimated_delivery_date,
    o.order_delivered_customer_date,
    oi.freight_value,
    oi.price,
    op.payment_type,
    op.payment_installments,
    op.payment_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN order_payments op ON o.order_id = op.order_id
WHERE o.order_delivered_customer_date IS NOT NULL;
"""

df_delay = pd.read_sql(query, conn)


In [ ]:
#===Convertendo datas para datetime===#
df_delay['order_purchase_timestamp'] = pd.to_datetime(df_delay['order_purchase_timestamp'])
df_delay['order_estimated_delivery_date'] = pd.to_datetime(df_delay['order_estimated_delivery_date'])
df_delay['order_delivered_customer_date'] = pd.to_datetime(df_delay['order_delivered_customer_date'])

#===Criando a variável alvo: 1 = pedido atrasado, 0 = no prazo===#
df_delay['is_late'] = df_delay['order_delivered_customer_date'] > df_delay['order_estimated_delivery_date']


**Por que escolhi essas variáveis?**

- `freight_value`: reflete a complexidade ou distância da entrega
- `price`: produtos mais caros podem ter processos logísticos diferentes
- `payment_installments`: pode indicar perfil de compra ou urgência
- `payment_value`: valor total da compra
- `distance_km`: representa o fator logístico mais direto

Todas são numéricas, limpas e ajudam a construir um modelo simples, mas significativo.


In [ ]:
#===Consulta SQL para pegar as coordenadas médias dos CEPs===#
query_dist = """
SELECT 
    o.order_id,
    gc.lat AS customer_lat,
    gc.lng AS customer_lng,
    gs.lat AS seller_lat,
    gs.lng AS seller_lng
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN sellers s ON oi.seller_id = s.seller_id
JOIN geo_avg gc ON c.customer_zip_code_prefix = gc.geolocation_zip_code_prefix
JOIN geo_avg gs ON s.seller_zip_code_prefix = gs.geolocation_zip_code_prefix
WHERE o.order_delivered_customer_date IS NOT NULL;
"""

df_coords = pd.read_sql(query_dist, conn)


In [ ]:
#===Função para calcular a distância entre dois pontos geográficos===#
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

#===Calculando distância cliente-vendedor para cada linha===#
df_coords['distance_km'] = df_coords.apply(
    lambda row: haversine(row['customer_lat'], row['customer_lng'], row['seller_lat'], row['seller_lng']),
    axis=1
)


In [ ]:
#===Unindo as distâncias com o DataFrame de modelagem===#
df_delay = df_delay.merge(df_coords[['order_id', 'distance_km']], on='order_id', how='left')

#===Removendo linhas com valores ausentes nas variáveis do modelo===#
df_delay = df_delay.dropna(subset=['freight_value', 'price', 'payment_installments', 'payment_value', 'distance_km'])


**Por que a remoção de valores ausentes (NaN) foi feita aqui e não na limpeza geral?**

A linha `df_delay.dropna(subset=[...])` só foi aplicada depois que selecionamos as colunas específicas do modelo.

> Isso porque faz mais sentido remover dados ausentes **quando já sabemos quais variáveis serão usadas**.

Se essa remoção tivese acontecido na etapa de limpeza geral, dados importantes poderiam ser excluidos de forma prematura, sem saber se eles seriam realmente necessários para a modelagem.

In [ ]:
#===Importando o modelo e ajustando com class_weight='balanced'===#
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

#===Selecionando as features e a variável alvo===#
features = df_delay[['freight_value', 'price', 'payment_installments', 'payment_value', 'distance_km']]
target = df_delay['is_late'].astype(int)

#===Separando os dados em treino e teste===#

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)


model = LogisticRegression(class_weight='balanced')  #-> Força o modelo a dar atenção à classe minoritária (atrasos)
model.fit(X_train, y_train)

#===Fazendo previsões no teste===#
y_pred = model.predict(X_test)

#===Avaliação com métricas de classificação===#
print(classification_report(y_test, y_pred))



### O que essas métricas significam:

- **Classe 0 = Entrega no prazo**
  - `precision 0.94`: entre os que o modelo previu como "no prazo", 94% estavam corretos
  - `recall 0.63`: o modelo conseguiu encontrar 63% das entregas no prazo

- **Classe 1 = Entrega com atraso**
  - `precision 0.09`: quando ele prevê atraso, só 9% realmente atrasaram → ainda erra bastante
  - `recall 0.48`: mas já consegue identificar **quase metade** dos pedidos que realmente atrasaram

- **Acurácia geral = 62%**
  - A precissão está boa, mas ainda existem problemas como o desbalanceamento dos dados

- **F1-score classe 1 = 0.16**
  - Reflete que o modelo ainda **tem dificuldade para acertar os atrasos**

---

**Possíveis desenvolvimentos:**
- Testar outros modelos (como Random Forest)
- Explorar técnicas como `SMOTE` ou `undersampling`
- Incluir mais variáveis logísticas ou de tempo (ex: tempo de aprovação)

## Subetapa 3 — Segmentação de Clientes

Nesta etapa, utilizei a técnica de clustering (agrupamento) para segmentar os clientes em grupos com comportamentos semelhantes.
- Para cada cliente único, calculamos:
  - O número total de pedidos realizados (`num_orders`)
  - O ticket médio (valor médio gasto por pedido - `avg_ticket`)
- Esses dois indicadores foram padronizados para evitar distorções causadas pela escala dos valores.
- Utilizamos o algoritmo **KMeans**, com 3 clusters definidos para identificar padrões distintos de consumo.

Essa segmentação é útil para definir perfis de clientes e sugerir estratégias de marketing para cada grupo.

In [ ]:
#===Consulta SQL para obter número de pedidos e ticket médio por cliente===#
query = """
SELECT 
    c.customer_unique_id,
    COUNT(o.order_id) AS num_orders,
    AVG(oi.price) AS avg_ticket
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_unique_id;
"""

df_segmentation = pd.read_sql(query, conn)

In [ ]:
#===Importando bibliotecas para clustering===#
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#===Selecionando colunas para o modelo de cluster===#
X = df_segmentation[['num_orders', 'avg_ticket']]

#===Padronizando os dados para evitar distorção por escala===#
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#===Aplicando o KMeans com 3 clusters===#
kmeans = KMeans(n_clusters=3, random_state=42)
df_segmentation['cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
#===Gráfico de dispersão para visualizar os grupos de clientes===#
plt.figure(figsize=(8,6))
sns.scatterplot(data=df_segmentation, x='num_orders', y='avg_ticket', hue='cluster', palette='tab10')
plt.title("Segmentação de Clientes por Ticket Médio e Nº de Pedidos")
plt.xlabel("Número de Pedidos")
plt.ylabel("Ticket Médio (R$)")
plt.tight_layout()
plt.show()

**Perfis identificados pelos clusters:**

- **Cluster 0 – Clientes Casuais:**  
  Baixo número de pedidos e ticket médio reduzido. Provavelmente são clientes que compraram uma vez e não retornaram.  
  ➤ *Ação sugerida:* campanhas de reengajamento (ex: cupons de desconto para 2ª compra).

- **Cluster 1 – Compradores Premium:**  
  Ticket médio alto, mas com poucos pedidos. Possuem maior poder aquisitivo, mas não se fidelizaram.  
  ➤ *Ação sugerida:* ofertas personalizadas, vantagens exclusivas e recomendação de produtos de maior valor.

- **Cluster 2 – Clientes Fiéis:**  
  Alto volume de pedidos com ticket médio intermediário. Representam o público mais engajado e consistente da base.  
  ➤ *Ação sugerida:* programas de fidelidade, benefícios progressivos, acesso antecipado a ofertas.

## Subetapa 4 — Análise de Satisfação

Identificar os principais fatores que influenciam a nota de avaliação (`review_score`) que o cliente fornece após a entrega.

- Relacionamos a avaliação do cliente com três variáveis:
  - **Tempo de entrega** (diferença entre data de compra e data de entrega)
  - **Valor do produto** (`price`)
  - **Categoria do produto**
- Utilizamos visualizações:
  - **Boxplots** para entender a distribuição da nota em relação ao tempo de entrega e ao valor
  - **Gráfico de calor (heatmap)** cruzando faixas de preço com faixas de tempo de entrega, mostrando os piores e melhores cenários


In [ ]:
#===Consulta SQL para trazer review_score, tempo de entrega, valor e categoria===#
query = """
SELECT 
    r.review_score,
    julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp) AS delivery_time,
    oi.price,
    p.product_category_name
FROM order_reviews r
JOIN orders o ON r.order_id = o.order_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE r.review_score IS NOT NULL;
"""

df_reviews = pd.read_sql(query, conn)


In [ ]:
#===Visualizando como o tempo de entrega varia conforme a nota dada pelo cliente===#
plt.figure(figsize=(12,6))
sns.boxplot(data=df_reviews, x='review_score', y='delivery_time')
plt.title("Distribuição do Tempo de Entrega por Nota de Avaliação")
plt.xlabel("Nota de Avaliação")
plt.ylabel("Tempo de Entrega (dias)")
plt.grid(True)
plt.tight_layout()
plt.show()

- Notas mais baixas (1 e 2) estão associadas a tempos de entrega mais altos.
- Quanto mais rápida a entrega, maior a chance de avaliação positiva.

In [ ]:
#===Visualizando se o valor do produto influencia a nota===#
plt.figure(figsize=(10,6))
sns.boxplot(data=df_reviews, x='review_score', y='price')
plt.title("Distribuição do Valor do Produto por Nota de Avaliação")
plt.xlabel("Nota de Avaliação")
plt.ylabel("Valor do Produto (R$)")
plt.grid(True)
plt.tight_layout()
plt.show()


- O preço do produto parece ter pouca influência com a nota.
- Clientes dão notas baixas tanto para compras baratas quanto compras caras, o tempo de entrega possui um peso maior.

In [ ]:
#===Criando faixas categóricas para visualização cruzada===#
df_reviews['delivery_range'] = pd.cut(df_reviews['delivery_time'], bins=[0,5,10,15,20,30,100], labels=['0–5','6–10','11–15','16–20','21–30','31+'])
df_reviews['price_range'] = pd.cut(df_reviews['price'], bins=[0,50,150,300,600,10000], labels=['até 50', '51–150', '151–300', '301–600', '601+'])

#===Criando tabela dinâmica com média das notas===#
pivot = df_reviews.pivot_table(index='delivery_range', columns='price_range', values='review_score', aggfunc='mean')

#===Plotando o gráfico de calor===#
plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Nota Média por Faixa de Entrega e Faixa de Preço")
plt.xlabel("Faixa de Preço (R$)")
plt.ylabel("Tempo de Entrega (dias)")
plt.tight_layout()
plt.show()


**Principais Insights:**

- O tempo de entrega é o fator com maior influência na avaliação.
- Mesmo produtos baratos têm nota alta se forem entregues rapidamente.
- As piores médias de nota ocorrem quando o produto é barato e demora muito para chegar (ex: "até 50 R$" entregues em "31+ dias").

**Ações sugeridas:**
- Focar em logística para produtos de menor valor
- Gerenciar melhor as expectativas de prazo com o cliente
- Monitorar categorias com histórico de demora + nota baixa